### Defining Random Variables in =python=                       :code_example:



In [1]:
# Define "tag" the display of which hides the cell
from IPython.display import HTML
from IPython.display import display

# Taken from https://stackoverflow.com/questions/31517194/how-to-hide-one-specific-cell-input-or-output-in-ipython-notebook
def tag(marker):
    s = HTML('''<script>
                  code_show=true; 
                  function code_toggle() {
                     if (code_show){
                        $('div.cell.code_cell.rendered.selected div.input').hide();
                     } else {
                        $('div.cell.code_cell.rendered.selected div.input').show();
                     }
                     code_show = !code_show
                   } 
                  $( document ).ready(code_toggle);
                 </script>
<a href="javascript:code_toggle()">%s</a>.''' % marker)
    return s
display(tag(''))

The distinguishing feature of variables in a field such as the
   reals or the complex plane is their *value*; the distinguishing
   feature of random variables is their *distribution*.  The `python`
   package `scipy.stats` is well-engineered and offers many different
   distributions, and tools to construct others, while the package
   `pacal` is perhaps less well engineered, but defines arithmetic
   operations over random variables which allows for more elegant
   semantics.

There are two main classes of random variables to consider: discrete
and continuous.  The distinction is worth drawing because different
classes are handled differently in many mathematical operations.  

For example, here we instantiate a scalar $\rvx$:



In [1]:
from scipy.stats import distributions as iid

x = iid.norm()

And here we instantiate a discrete random variable which is defined
 over an event space $\{-1,0,1\}$ with corresponding probabilities $(1/3,1/2,1/6)$:



In [1]:
Omega = (-1,0,1)
Pr = (1/3.,1/2.,1/6.)

s = iid.rv_discrete(values=(Omega,Pr))

Now, here are some things we can do with these random variables.
 First, the continuous  $\rvx$:



In [1]:
print("E(x) = %6.4f" % x.mean())
print()
print("Some (central) moments of x:")
print([(m,x.moment(m)) for m in [1,2,3,4]])
print()
print("95%% confidence interval: (%f,%f)" % x.interval(0.95))
print()
print(x.cdf(0),x.pdf(0))

:results:
 E(x) = 0.0000

 Some (central) moments of x:
 [(1, 0.0), (2, 1.0), (3, 0.0), (4, 3.0)]

 95% confidence interval: (-1.959964,1.959964)

 0.5 0.3989422804014327
 :end:

Next, the discrete r.v., \rv{s}:



In [1]:
print("E(s) = %6.4f" % s.mean())
print()
print("Some moments of x:")
print([(m,s.moment(m)) for m in [1,2,3,4]])
print()
print("95%% confidence interval: (%f,%f)" % s.interval(0.95))
print()
# Note! Not pdf, but pmf for discrete rv.
print(s.cdf(0),s.pmf(0))

If we want *realizations* of these random variables:



In [1]:
N=3
print(x.rvs(N)) # N realizations; no longer random

We&rsquo;d like to be able to combine different random variables, say
by addition, yielding a new random variable.  For instance, we&rsquo;d like
to be able to construct



In [1]:
y = x + s

But this fails.  Can you explain why?  What do you suppose the cdf of
$\rvy$ looks like?  Does it have a density, or does the addition of a
random variable that *lacks* a density ($\rv{s}$) to a random variable
that has one ($\rvx$) mess things up?



In [1]:
display(tag("+")
# Code to convolve a random variable with a pmf and another having a cdf
# Exploits =scipy.stats= base rv_continuous class.

class ConvolvedContinuousAndDiscrete(iid.rv_continuous):

    """Convolve (add) a continuous rv x and a discrete rv s,
       returning the resulting cdf."""

    def __init__(self,f,s):
        self.continuous_rv = f
        self.discrete_rv = s
        super(ConvolvedContinuousAndDiscrete, self).__init__(name="ConvolvedContinuousAndDiscrete")
        
    def _cdf(self,z):
        F=0
        s = self.discrete_rv
        x = self.continuous_rv
        
        for k in range(len(s.xk)):
            F = F + x.cdf(z-s.xk[k])*s.pk[k]
        return F

    def _pdf(self,z):
        f=0
        s = self.discrete_rv
        x = self.continuous_rv
        
        for k in range(len(s.xk)):
            f = f + x.pdf(z-s.xk[k])*s.pk[k]
        return f


# Create new convolved rv:
y = ConvolvedContinuousAndDiscrete(x,s)

In [1]:
import plotly.graph_objects as go
import numpy as np

X = np.linspace(-4,4,100).tolist()

fig = go.Figure(data=go.Scatter(x=X, y=[y.pdf(z) for z in X]))
fig.show()

#### Exercise



Prove that $\rvy$ is continuous (in the sense that it has a density),
     as suggested by the figure *or* establish that the figure is
     wrong or misleading.



#### Proof



Let $F_x$ denote the cdf of \rvx.  We want to establish that the cdf
of $\rvy$, say $F_y(y)=\Pr(\rvy\leq y)$ is a continuously differentiable
function of $y$.  We use the fact that the distribution of $\rvy$ is a
convolution of $\rvx$ and $\rvs$, so that

\begin{equation} 
\begin{split}
    \Pr(\rvy\leq y) &= \Pr(\rv{s} + \rv{x}\leq y ) \\
                    &= \sum_{s\in\Omega}\Pr(\rvx\leq y-s|s)\pi_s\\
                    &= \sum_{s\in\Omega}F_x(y-s)\pi_s,
\end{split}
\end{equation}

which is continuously differentiable in $y$, as required.

